# How to run pygrass `Modules` in parallel

It is possible to run `Modules` object in parallel. There are more ways, in this documents there are two of them.

* using `subprocess` module
* using `ParallelModuleQueue` pygrass modules

## subprocess

## ParallelModuleQueue

# Summary

# Exercise